In [1]:
from typing import List

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

import cornac
from cornac.data import Dataset
from cornac.eval_methods import RatioSplit
from cornac.exception import ScoreException
from cornac.metrics import AUC, MAE, MAP, NDCG, RMSE, Precision, Recall
from cornac.models import BPR, MF, PMF, MostPop, Recommender
from sklearn.base import RegressorMixin

class SimpleEnsemble(Recommender):
    def __init__(self, models:List[Recommender]=None, meta_learner=None,  *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.models : List[Recommender] = models
        self.meta_learner = meta_learner

    def fit(self, train_set:Dataset, val_set:Dataset=None):
        Recommender.fit(self, train_set, val_set)

        uir = np.array(train_set.uir_tuple).T
        print(uir)
        # df_uir:pd.DataFrame = pd.DataFrame(uir[:,[0,1]]).astype(int)
        # df_uir=df_uir.groupby(0).aggregate(lambda x: list(map(int,x)))[1]
        
        # user_items = df_uir.to_dict()
        # print(df_uir)
        # print(user_items)
        # print(df_uir.to_dict())
        
        models_scores=[]
        for model in self.models:
            model.fit(train_set,val_set)
            scores= []
            for u,i,r in train_set.uir_iter():
                score = model.score(u[0],i[0])
                scores.append(score)
                
            # scores = model.score(uir[:,0].astype(int),uir[:,1].astype(int))
            models_scores.append(scores)
        models_scores = np.array(models_scores).T
        self.models_scores_normalizer = StandardScaler()
        models_scores = self.models_scores_normalizer.fit_transform(models_scores)
        self.meta_learner.fit(models_scores, uir[:,2])
        # self.item_pop = np.ediff1d(train_set.csc_matrix.indptr)
        return self
    def score(self, user_idx, item_idx=None):
        if item_idx is None:
            models_scores = []
            for model in self.models:
                scores = model.score(user_idx)
                models_scores.append(scores)
            models_scores = np.array(models_scores).T
            models_scores = self.models_scores_normalizer.transform(models_scores)
            predicted = self.meta_learner.predict(models_scores)
            # print(predicted)
            return predicted
        else:
            models_scores = []
            for model in self.models:
                score = model.score(user_idx,item_idx)
                models_scores.append([score])
            models_scores = np.array(models_scores).T
            models_scores = self.models_scores_normalizer.transform(models_scores)
            predicted = self.meta_learner.predict(models_scores)
            # print(predicted)
            return predicted[0]

# load the built-in MovieLens 100K and split the data based on ratio
ml_100k = cornac.datasets.movielens.load_feedback()
rs = RatioSplit(data=ml_100k, test_size=0.2, rating_threshold=4.0, seed=123)

# initialize models, here we are comparing: Biased MF, PMF, and BPR
models = [
    SimpleEnsemble(models=[MostPop(),MF(k=10, max_iter=25, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123)],meta_learner=MLPRegressor(),name='SimpleEnsemble'),
    MF(k=10, max_iter=25, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123),
    # PMF(k=10, max_iter=100, learning_rate=0.001, lambda_reg=0.001, seed=123),
    # BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123),
]

# define metrics to evaluate the models
metrics = [MAE(), RMSE(), Precision(k=10), Recall(k=10), NDCG(k=10), AUC(), MAP()]

# put it together in an experiment, voilà!
cornac.Experiment(eval_method=rs, models=models, metrics=metrics, user_based=True).run()


[[  0.   0.   4.]
 [  1.   1.   5.]
 [  2.   2.   5.]
 ...
 [198.  30.   5.]
 [ 69. 429.   4.]
 [427. 644.   4.]]

TEST:
...
               |    MAE |   RMSE |    AUC |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------------- + ------ + ------ + ------ + ------ + ------- + ------------ + --------- + --------- + --------
SimpleEnsemble | 0.7459 | 0.9081 | 0.7137 | 0.0338 |  0.0380 |       0.0358 |    0.0281 |   34.6070 |  36.1721
MF             | 0.7430 | 0.8998 | 0.7445 | 0.0407 |  0.0479 |       0.0437 |    0.0352 |    0.1592 |   3.9834

